# Auth

> Client side logic to add Plash Auth to your app

For an end2end example on how to use this module see `./examples/auth/main.py`

In [ ]:
#| default_exp auth

In [ ]:
#| export
import httpx,os,jwt
from pathlib import Path
from warnings import warn

from plash_cli import __version__

The signin completion route is where Google redirects users after authentication. Your app needs to add this route to handle the OAuth callback.

In [ ]:
#| export
signin_completed_rt = "/signin_completed"

The production flag lets developers use mock authentication during development.

In [ ]:
#| export
_in_prod = os.getenv('PLASH_PRODUCTION', '') == '1'

This function makes the actual HTTP request to the Plash authentication service. It sends the email and domain filters to get back a signin URL and request ID that we'll use to track this authentication attempt.

In [ ]:
#| export
def _plash_signin_url(email_re: str=None, hd_re: str=None):
    res = httpx.post(os.environ['PLASH_AUTH_URL'], json=dict(email_re=email_re, hd_re=hd_re), 
                     auth=(os.environ['PLASH_APP_ID'], os.environ['PLASH_APP_SECRET']), 
                     headers={'X-PLASH-AUTH-VERSION': __version__}).raise_for_status().json()
    if "warning" in res: warn(res.pop('warning'))
    return res

This is the main function your app calls to get a Google signin URL. In development mode, it returns a mock URL to make testing easier. In production, it calls the Plash auth service and stores the request ID in the session for later verification.

In [ ]:
#| export
def mk_plash_signin_url(session: dict,       # Session dictionary
                        email_re: str=None,  # Regex filter for allowed email addresses
                        hd_re: str=None):    # Regex filter for allowed Google hosted domains
    "Generate a Google Sign-In URL for Plash authentication."
    if not _in_prod: return f"{signin_completed_rt}?signin_reply=mock-sign-in-reply"
    res = _plash_signin_url(email_re, hd_re)
    session['req_id'] = res['req_id']
    return res['plash_signin_url']

After Google authentication, Plash sends back a JSON Web Token (JWT) containing the user's information. This function decodes and validates that token using the ES256 public key. If anything goes wrong with the JWT, it returns error details instead of crashing.

Note: a JWT does not mean the message is encrypted. It ensures data integrity and authenticity, it protects against tampering and forgery. We use JWT tokens so your app can trust that the sign-in information and user details it receives after authentication really come from Plash (and by extension, Google), and have not been modified by an attacker.

In [ ]:
#| export
def _parse_jwt(reply: str) -> dict:
    "Parse JWT reply and return decoded claims or error info"
    try: decoded = jwt.decode(reply, key=open(Path(__file__).parent / "assets" / "es256_public_key.pem","rb").read(), algorithms=["ES256"], 
                              options=dict(verify_aud=False, verify_iss=False))
    except Exception as e: return dict(req_id=None, valid=False, sub=None, err=f'JWT validation failed: {e}')
    return dict(req_id=decoded.get('req_id'), valid=True, sub=decoded.get('sub'), err=decoded.get('err'))

A custom exception for when authentication fails. This makes it easier for your app to handle auth errors specifically.
See `./examples/auth/main.py` for an example on how you can catch this exception in your application.

In [ ]:
#| export
class PlashAuthError(Exception):
    """Raised when Plash authentication fails"""
    pass

This is the main function your app calls in the signin completion route. It verifies the JWT reply matches the original request (preventing CSRF attacks), checks for any authentication errors, and returns the user's Google ID if everything is valid.

When testing locally this will always return the mock Google ID `'424242424242424242424'`.

In [ ]:
#| export
def goog_id_from_signin_reply(session: dict, # Session dictionary containing 'req_id'
                              reply: str):   # The JWT reply string from Plash after Google authentication
    "Validate Google sign-in reply and returns Google user ID if valid."
    if not _in_prod: return '424242424242424242424'
    parsed = _parse_jwt(reply)
    if session['req_id'] != parsed['req_id']: raise PlashAuthError("Request originated from a different browser than the one receiving the reply")
    if parsed['err']: raise PlashAuthError(f"Authentication failed: {parsed['err']}")
    return parsed['sub'] if parsed['valid'] else None

## Export -

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()